In [2]:
from pathlib import Path

import numpy as np
import pyrofit.lensing
import torch
from imageio import imsave
from matplotlib import pyplot as plt
from pyrofit.lensing.lenses import ExternalShear, SPLELens
from pyrofit.lensing.model import LensingModelBase
from pyrofit.lensing.sources import ImageSource
from pyrofit.lensing.utils import get_meshgrid
from ruamel.yaml import YAML
from skimage.measure import find_contours
from skimage.transform import downscale_local_mean

from clipppy.stochastic import stochastic

In [25]:
PAPERDIR = Path().absolute().parent.parent
GALLERYDIR = PAPERDIR / 'tikz' / 'gallery'
OBJECTDATADIR = PAPERDIR / 'tikz' / 'object-data'

In [26]:
PAPERDIR, GALLERYDIR, OBJECTDATADIR

(PosixPath('/home/eliasd/lensing/pyrofit-lensing-analysis'),
 PosixPath('/home/eliasd/lensing/pyrofit-lensing-analysis/tikz/gallery'),
 PosixPath('/home/eliasd/lensing/pyrofit-lensing-analysis/tikz/object-data'))

In [28]:
Path(pyrofit.lensing.__path__[0]).parent.parent

PosixPath('/home/eliasd/lensing/pyrofit_lensing')

In [22]:
SYSTEM_NAME = 'hoags_object'
PLDIR = Path(pyrofit.lensing.__path__[0]).parent.parent
BASEDIR = PLDIR / 'experiments' / 'kk'
MOCKDIR = PLDIR / 'mock_data'
# PARAMS = YAML().load(BASEDIR / 'params.yaml')[SYSTEM_NAME]
PARAMS = YAML().load(open('../paper-source/params.yaml'))[SYSTEM_NAME]

In [21]:
PARAMS

ordereddict([('scale', 1.0), ('truth', ordereddict([('ext/gamma_1', -0.0227), ('ext/gamma_2', 0.0047), ('main/phi', 1.742), ('main/q', 0.2753), ('main/r_ein', 1.1847), ('main/slope', 2.2193), ('main/x', -0.0704), ('main/y', -0.0193)])), ('init', ordereddict([('ext/gamma_1', 0.0), ('ext/gamma_2', 0.0), ('main/phi', 1.5), ('main/q', 0.3), ('main/r_ein', 1.2), ('main/slope', 2.0), ('main/x', 0.0), ('main/y', 0.0)]))])

In [23]:
SCALE = 5
RES = (640, 640)
SUBSAMPLING = 10
SNR = 30

In [30]:
SOURCE_DIR = '../../mock_data/sources'
SOURCE_DIR

'../../mock_data/sources'

In [31]:
X, Y = get_meshgrid(SCALE/RES[0]/SUBSAMPLING, RES[0]*SUBSAMPLING, RES[1]*SUBSAMPLING)
model = LensingModelBase(X, Y, alphas={
    'main': stochastic(SPLELens(), {}), 'ext': stochastic(ExternalShear(), {})
}, sources={
    'img': ImageSource(f'{SYSTEM_NAME}.jpg',
                       scale=PARAMS['scale'], origin='upper')
})
isrc = model.sources['img']

TypeError: 'module' object is not callable

In [1]:
for param, val in PARAMS['truth'].items():
    a, b = param.split('/', 1)
    model.alphas[a].stochastic_specs[b] = val

img = downscale_local_mean(torch.movedim(model(SUBSAMPLING/SNR,), -3, -1), (SUBSAMPLING, SUBSAMPLING, 1))

imag = model.inv_magnification()
crit = (
    (max(find_contours(imag.numpy(), level=0), key=len) / np.array(model.Xmid.shape) - 0.5)
    * np.array((model.Ymid[-1, 0] - model.Ymid[0, 0], model.Xmid[0, -1] - model.Xmid[0, 0]))
)[..., ::-1]
caus = model.raytrace_arbitrary(torch.from_numpy(crit.copy()).to(torch.get_default_dtype())).numpy()


imkwargs = dict(origin='lower', extent=np.array((-1, 1, -1, 1)) * SCALE / 2)

# Origin when plotting is tied to the X,Y grid, so always lower!
plt.imshow(img, **imkwargs)
plt.plot(*crit.T, 'r:')
# plt.gca().add_artist(matplotlib.patches.Ellipse((m.x, m.y), 2*m.r_ein/sqrt(m.q), 2*m.r_ein*sqrt(m.q), np.rad2deg(m.phi),
#                                                 fill=False, ls='--', color='green'))

imsave(GALLERYDIR / f'{SYSTEM_NAME}-img-color.jpg', (np.clip(img[::-1], 0, 1)*255).astype(np.uint8))
np.savetxt(OBJECTDATADIR / SYSTEM_NAME / f'{SYSTEM_NAME}-critical.txt', crit)
np.savetxt(OBJECTDATADIR / SYSTEM_NAME / f'{SYSTEM_NAME}-caustic.txt', caus)

FileNotFoundError: [Errno 2] No such file or directory: '/home/eliasd/lensing/pyrofit_lensing/experiments/kk/params.yaml'